In [1]:
import torch
import numpy as np
import pandas as pd
import random as rd
import losses as ls
import csv

In [2]:
# default stuff
dtype = torch.double
device = torch.device("cpu")

# architecture definition
N_in, N_hid, N_out = 2, 10, 4
batch_size = 200
learning_rate = 1e-6

#weight matrix
w1 = torch.randn(N_in, N_hid, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(N_hid, N_out, device=device, dtype=dtype, requires_grad=True)

In [3]:
# data manipulation
# ho cambiato il file così ora sono 0 e 1 i colori
data = pd.read_csv('/home/burrito/Documents/python/pytorch/Python-net/color_classifier/data_train_4.csv')
test = pd.read_csv('/home/burrito/Documents/python/pytorch/Python-net/color_classifier/data_test_4.csv')
data_np_x = data[['x', 'y']].to_numpy()
data_np_y = data[['color']].to_numpy()
data_test = test.to_numpy()

In [4]:
batchesX = np.asarray(np.array_split(data_np_x, 100))
batchesY = np.asarray(np.array_split(data_np_y, 100))

In [5]:
Xtr = torch.from_numpy(batchesX[2])
Ytr = torch.from_numpy(batchesY[2])
Xtest = torch.from_numpy(data_test)

In [6]:
for epoch in range(100000):
    b = rd.randint(0, 99)
    Xtr = torch.from_numpy(batchesX[b])
    Ytr = torch.from_numpy(batchesY[b])
    y = Xtr.mm(w1).clamp(min=0).mm(w2)
    # SVM loss
    # loss = ls.SVM_loss(y, Ytr)
    # cross entropy loss
    loss = ls.cross_entropy(y, Ytr)
    if epoch%10000 == 0:
        print(epoch, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0 171.60410034484786
10000 151.43581482230152
20000 152.27593746106749
30000 142.4808454497628
40000 137.5842834586582
50000 136.30890297423377
60000 138.0060337070858
70000 132.12448236683028
80000 135.65550836959648
90000 132.56309186301627


In [7]:
prediction = Xtest.mm(w1).clamp(min=0).mm(w2)

In [8]:
pred = open("/home/burrito/Documents/python/pytorch/Python-net/color_classifier/pred_4.csv", 'w+', newline='')
writer = csv.writer(pred)
writer.writerow(["color"])
for i in prediction:
    value, index = torch.max(i, 0)
    writer.writerow([index])